# lab-12-6 sequence to sequence with attention (Keras + eager version)

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
* [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025)
* [Neural Machine Translation with Attention from Tensorflow](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb)

In [47]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os
import pandas as pd
print(tf.__version__)

1.15.0


In [102]:
import re

In [166]:
df = pd.read_json('moviedata_preprocessed.json')

In [167]:
df.drop(columns = 'level_0', axis= 1, inplace = True)

In [168]:
df = df.reset_index()
df

,level_0,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
0,0,0,"[Adventure, Drama, Horror, Thriller]",47 Meters Down: Uncaged,www.imdb.com/title/tt7329656/mediaviewer/rm242...,four teen girls diving in a ruined underwater ...,"[The film starts at a girls school in Yucatan,...",the next chapter
1,1,2,"[Horror, Sci-Fi]",Alien,www.imdb.com/title/tt0078748/mediaviewer/rm299...,after a space merchant vessel perceives an unk...,[The opening credits appear in front of a larg...,its alien the 8th passenger.
2,10,1854,"[Horror, Sci-Fi]",Revenge of the Creature,www.imdb.com/title/tt0048554/mediaviewer/rm246...,men capture the creature from the black lagoon...,[A tributary of the upper Amazon. A small boat...,allnew 3d thills original poster
3,100,2011,"[Biography, Drama, Music]",Eden,www.imdb.com/title/tt3090634/mediaviewer/rm836...,paul a teenager in the underground scene of ea...,"[\n , \n ]",the sun rises on every party
4,1000,3683,[Western],God Forgives... I Don't!,www.imdb.com/title/tt0061576/mediaviewer/rm171...,after a train is robbed of its payroll an insu...,"[\n , \n ]",how many men must die
5,1001,44,"[Action, Adventure, Sci-Fi, Thriller]",The Hunger Games: Mockingjay - Part 1,www.imdb.com/title/tt1951265/mediaviewer/rm425...,katniss everdeen is in district 13 after she s...,"[Katniss Everdeen (, ), victor of the 74th and...",fire burns brighter in the darkness
6,1002,368,[Horror],Veronica,www.imdb.com/title/tt5862312/mediaviewer/rm175...,madrid 1991. a teen girl finds herself besiege...,"[\n , \n ]",alguien respondi a tu llamada. someone answere...
7,1003,3684,"[Comedy, Romance]",Hero at Large,www.imdb.com/title/tt0080863/mediaviewer/rm275...,an idealistic but struggling actor finds his l...,"[\n , \n ]",help is on the way
8,1004,3686,"[Action, Crime, Drama, Thriller]",One Good Cop,www.imdb.com/title/tt0102593/mediaviewer/rm588...,an nypd detective michael keaton and his strug...,"[\n , \n ]",they killed his partner threatened his family....
9,1005,3689,"[Horror, Mystery, Sci-Fi, Thriller]",Strange Behavior,www.imdb.com/title/tt0082243/mediaviewer/rm118...,a scientist is experimenting with teenagers an...,"[\n , \n ]",murder mayhem and mystery strikes fear into a ...


In [169]:

df.movie_name = df.movie_name.apply(lambda x: x.replace('혻',''))

df.movie_summary = df.movie_summary.apply(lambda x: str(x).lower())
df.movie_summary = df.movie_summary.apply(lambda x: re.sub("[^A-Za-z. 0-9]", '', x))
df.movie_summary = df.movie_summary.apply(lambda x: x.replace('\\n',''))

df.movie_tagline = df.movie_tagline.apply(lambda x: str(x).lower())
df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('\\n','').replace('\\xa0',''))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub('[^A-Za-z. 0-9]', '', x))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub(' {2,}', '', x))

#태그라인 없는 데이터 row 삭제
index = df[df['movie_tagline']==''].index
df.drop(index, inplace = True)
df.drop(columns = 'level_0', axis= 1, inplace = True)
#summary 없는 데이터 row 삭제
summary_index = df[df['movie_summary']==''].index
df.drop(summary_index, inplace = True)

index = df[df['movie_summary'].str.contains('  ')].index
df.drop(index , inplace = True)




8636
8636
8636


In [137]:
df.to_json('moviedata_preprocessed.json')

In [133]:

sources = [w.split() for w in df.movie_summary]
targets = [w.split() for w in df.movie_tagline]

In [180]:
sources[0]

['four',
 'teen',
 'girls',
 'diving',
 'in',
 'a',
 'ruined',
 'underwater',
 'city',
 'quickly',
 'learn',
 'theyve',
 'entered',
 'the',
 'territory',
 'of',
 'the',
 'deadliest',
 'shark',
 'species',
 'in',
 'the',
 'claustrophobic',
 'labyrinth',
 'of',
 'submerged',
 'caves.',
 '47',
 'meters',
 'down',
 'uncaged',
 'follows',
 'the',
 'diving',
 'adventure',
 'of',
 'four',
 'teenage',
 'girls',
 'sophie',
 'nlisse',
 'corinne',
 'foxx',
 'brianne',
 'tju',
 'and',
 'sistine',
 'stallone',
 'exploring',
 'a',
 'submerged',
 'mayan',
 'city.',
 'once',
 'inside',
 'their',
 'rush',
 'of',
 'excitement',
 'turns',
 'into',
 'a',
 'jolt',
 'of',
 'terror',
 'as',
 'they',
 'discover',
 'the',
 'sunken',
 'ruins',
 'are',
 'a',
 'hunting',
 'ground',
 'for',
 'deadly',
 'great',
 'white',
 'sharks.',
 'with',
 'their',
 'air',
 'supply',
 'steadily',
 'dwindling',
 'the',
 'friends',
 'must',
 'navigate',
 'the',
 'underwater',
 'labyrinth',
 'of',
 'claustrophobic',
 'caves',
 'an

In [135]:
# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}

pprint(source2idx)

{'.': 1,
 '..': 2,
 '...': 3,
 '....': 4,
 '...darren': 5,
 '...murder.': 6,
 '...or': 7,
 '...well': 8,
 '..until': 9,
 '.45': 10,
 '.45.': 11,
 '.and': 12,
 '.it': 13,
 '.jai': 14,
 '.nevertheless': 15,
 '.next': 16,
 '.sonia': 17,
 '.they': 18,
 '0': 19,
 '0.': 20,
 '000.': 21,
 '00000': 22,
 '007.': 23,
 '04': 24,
 '06': 25,
 '1': 26,
 '1.': 27,
 '1.5': 28,
 '1.8': 29,
 '10': 30,
 '10.': 31,
 '100': 32,
 '1000': 33,
 '10000': 34,
 '10000.': 35,
 '10000.00': 36,
 '100000': 37,
 '100000.': 38,
 '100000.00': 39,
 '1000000': 40,
 '1000000.': 41,
 '1000th': 42,
 '1001': 43,
 '1007': 44,
 '100foottall': 45,
 '100point': 46,
 '100th': 47,
 '100yearold': 48,
 '104': 49,
 '1087': 50,
 '10m': 51,
 '10th': 52,
 '10thcentury': 53,
 '10year': 54,
 '10yearold': 55,
 '11': 56,
 '111': 57,
 '1114': 58,
 '112': 59,
 '114': 60,
 '1145': 61,
 '117': 62,
 '11pm.': 63,
 '11th': 64,
 '11year': 65,
 '11yearold': 66,
 '12': 67,
 '120': 68,
 '120.': 69,
 '1200': 70,
 '12000': 71,
 '1200s': 72,
 '1214': 73,

 'accordion': 1282,
 'accordion.': 1283,
 'accords.': 1284,
 'accosted': 1285,
 'accosting': 1286,
 'account': 1287,
 'account.': 1288,
 'accountability': 1289,
 'accountable': 1290,
 'accountant': 1291,
 'accountant.': 1292,
 'accounting': 1293,
 'accounts': 1294,
 'accounts.': 1295,
 'accouterments': 1296,
 'accredited': 1297,
 'accross': 1298,
 'accumulates': 1299,
 'accumulator': 1300,
 'accuracy': 1301,
 'accuracy.': 1302,
 'accurate': 1303,
 'accurate.': 1304,
 'accurately': 1305,
 'accursed': 1306,
 'accusation': 1307,
 'accusations': 1308,
 'accusations.': 1309,
 'accuse': 1310,
 'accused': 1311,
 'accused.': 1312,
 'accuses': 1313,
 'accusing': 1314,
 'accussed': 1315,
 'accustomed': 1316,
 'ace': 1317,
 'ace.': 1318,
 'acerbic': 1319,
 'aces': 1320,
 'achieve': 1321,
 'achieved': 1322,
 'achieved.': 1323,
 'achievement': 1324,
 'achievements': 1325,
 'achievements.': 1326,
 'achieves': 1327,
 'achieving': 1328,
 'achille': 1329,
 'achilles': 1330,
 'achim': 1331,
 'aching': 1

 'ambiguity.': 2532,
 'ambiguous': 2533,
 'ambiguous.': 2534,
 'ambition': 2535,
 'ambition.': 2536,
 'ambitionless': 2537,
 'ambitions': 2538,
 'ambitions.': 2539,
 'ambitionsto': 2540,
 'ambitious': 2541,
 'ambitious.': 2542,
 'ambitiously': 2543,
 'ambivalence': 2544,
 'ambivalence.': 2545,
 'ambivalent': 2546,
 'ambivalent.': 2547,
 'ambrose': 2548,
 'ambrosia.': 2549,
 'ambulance': 2550,
 'ambulance.': 2551,
 'ambulancechasing': 2552,
 'ambulances': 2553,
 'ambush': 2554,
 'ambush.': 2555,
 'ambush.but': 2556,
 'ambushed': 2557,
 'ambushed.': 2558,
 'ambushes': 2559,
 'ambushsurviving': 2560,
 'amc': 2561,
 'ameer': 2562,
 'ameila.': 2563,
 'amelia': 2564,
 'amelia.': 2565,
 'amelias': 2566,
 'amelie': 2567,
 'ameliorating': 2568,
 'amenazas': 2569,
 'amendment': 2570,
 'amendments': 2571,
 'amends': 2572,
 'amenities.': 2573,
 'amenities...and': 2574,
 'amercan': 2575,
 'america': 2576,
 'america.': 2577,
 'americabound': 2578,
 'americal': 2579,
 'american': 2580,
 'american.': 

 'asit': 3781,
 'ask': 3782,
 'ask.': 3783,
 'asked': 3784,
 'asked.': 3785,
 'askew': 3786,
 'askew.': 3787,
 'asking': 3788,
 'asks': 3789,
 'askwith': 3790,
 'asleep': 3791,
 'asleep.': 3792,
 'asmodeus': 3793,
 'asner': 3794,
 'asolo': 3795,
 'aspect': 3796,
 'aspects': 3797,
 'aspects.': 3798,
 'aspen': 3799,
 'asperger': 3800,
 'aspergers': 3801,
 'aspersion': 3802,
 'asphalt': 3803,
 'aspirant': 3804,
 'aspiration': 3805,
 'aspirationat': 3806,
 'aspirations': 3807,
 'aspire': 3808,
 'aspires': 3809,
 'aspires.': 3810,
 'aspiring': 3811,
 'asrod': 3812,
 'asrod.': 3813,
 'ass': 3814,
 'assailant': 3815,
 'assailant.': 3816,
 'assailants': 3817,
 'assailants.': 3818,
 'assaination': 3819,
 'assam': 3820,
 'assasinated': 3821,
 'assasination': 3822,
 'assasinationturnedsuicide': 3823,
 'assassin': 3824,
 'assassin.': 3825,
 'assassinate': 3826,
 'assassinated': 3827,
 'assassinated.': 3828,
 'assassinates': 3829,
 'assassinating': 3830,
 'assassination': 3831,
 'assassination.': 3

 'bars...': 5031,
 'barsha': 5032,
 'bart': 5033,
 'bartell.': 5034,
 'bartender': 5035,
 'bartenders': 5036,
 'bartenderwine': 5037,
 'barter': 5038,
 'barterbride': 5039,
 'barters': 5040,
 'bartha': 5041,
 'bartholomew': 5042,
 'bartle': 5043,
 'bartleby': 5044,
 'bartleby.': 5045,
 'bartlett': 5046,
 'barto': 5047,
 'bartog': 5048,
 'bartolo': 5049,
 'barton': 5050,
 'bartons': 5051,
 'baryshnikov': 5052,
 'barzell': 5053,
 'basa': 5054,
 'basa.': 5055,
 'basanite': 5056,
 'basas': 5057,
 'bascomb': 5058,
 'base': 5059,
 'base.': 5060,
 'baseball': 5061,
 'baseball.': 5062,
 'baseballlistening': 5063,
 'baseboard': 5064,
 'based': 5065,
 'based.': 5066,
 'basehart': 5067,
 'basement': 5068,
 'basement.': 5069,
 'basements': 5070,
 'baser': 5071,
 'bases': 5072,
 'bases.': 5073,
 'bash': 5074,
 'bash.': 5075,
 'bashed': 5076,
 'basher': 5077,
 'bashful': 5078,
 'bashing': 5079,
 'bashir': 5080,
 'basic': 5081,
 'basically': 5082,
 'basicallysimple': 5083,
 'basics': 5084,
 'basil': 

...

 'they': 52293,
 'theyd': 52294,
 'theyll': 52295,
 'theyre': 52296,
 'theyve': 52297,
 'thick': 52298,
 'thickened': 52299,
 'thickens': 52300,
 'thickens.': 52301,
 'thicker': 52302,
 'thicket': 52303,
 'thickheaded': 52304,
 'thickwitted': 52305,
 'thief': 52306,
 'thief.': 52307,
 'thief2san': 52308,
 'thiefbooster': 52309,
 'thiefs': 52310,
 'thier': 52311,
 'thieriot': 52312,
 'thierry': 52313,
 'thierrys': 52314,
 'thiers': 52315,
 'thievery': 52316,
 'thieves': 52317,
 'thieves.': 52318,
 'thieves...': 52319,
 'thieveshowever': 52320,
 'thieving': 52321,
 'thin': 52322,
 'thin.': 52323,
 'thing': 52324,
 'thing.': 52325,
 'things': 52326,
 'things.': 52327,
 'things...': 52328,
 'thingy': 52329,
 'think': 52330,
 'think.': 52331,
 'think...': 52332,
 'thinker': 52333,
 'thinkers': 52334,
 'thinking': 52335,
 'thinks': 52336,
 'thinks.': 52337,
 'thinks...and': 52338,
 'thinley': 52339,
 'thinly': 52340,
 'thinlydisguised': 52341,
 'thir': 52342,
 'third': 52343,
 'third.': 5234

 'tried.': 53635,
 'tried...': 53636,
 'trier': 53637,
 'triers': 53638,
 'tries': 53639,
 'trieste': 53640,
 'trifle': 53641,
 'trifle.': 53642,
 'trigger': 53643,
 'triggered': 53644,
 'triggered.': 53645,
 'triggerhappy': 53646,
 'triggering': 53647,
 'triggers': 53648,
 'trigorin': 53649,
 'trigorin.': 53650,
 'trilobite.': 53651,
 'trilobites.': 53652,
 'trilogy': 53653,
 'trina': 53654,
 'trina.': 53655,
 'trinas': 53656,
 'trinas.': 53657,
 'trine': 53658,
 'trinidad': 53659,
 'trinity': 53660,
 'trinkets': 53661,
 'trio': 53662,
 'trio.': 53663,
 'trios': 53664,
 'trip': 53665,
 'trip.': 53666,
 'trip...': 53667,
 'tripathi.': 53668,
 'tripathis': 53669,
 'tripathis.': 53670,
 'triple': 53671,
 'triplemurder': 53672,
 'triplet': 53673,
 'triplets': 53674,
 'triplike': 53675,
 'tripod': 53676,
 'tripoli': 53677,
 'tripoli.': 53678,
 'tripp': 53679,
 'tripped': 53680,
 'tripper': 53681,
 'tripper.': 53682,
 'trippin': 53683,
 'tripping': 53684,
 'trippy': 53685,
 'trips': 53686,


 'upended': 54965,
 'upended.': 54966,
 'upends': 54967,
 'upgraded': 54968,
 'upgraded.': 54969,
 'upheaval': 54970,
 'upheavals': 54971,
 'upheavals.': 54972,
 'uphill': 54973,
 'uphold': 54974,
 'upholding': 54975,
 'upholds': 54976,
 'uplift': 54977,
 'uplifting': 54978,
 'upliftment': 54979,
 'upload': 54980,
 'uploaded': 54981,
 'upmarket': 54982,
 'upon': 54983,
 'upper': 54984,
 'upperbourgeois': 54985,
 'upperclass': 54986,
 'upperclassman': 54987,
 'upperclassmen': 54988,
 'uppercrust': 54989,
 'upperstuffy': 54990,
 'upping': 54991,
 'upright': 54992,
 'uprightness': 54993,
 'uprising': 54994,
 'uprising.': 54995,
 'uprisings': 54996,
 'uprisings.': 54997,
 'upriver': 54998,
 'uproar': 54999,
 'uprooted': 55000,
 'uprooted.': 55001,
 'uproots': 55002,
 'ups': 55003,
 'upscale': 55004,
 'upscaling': 55005,
 'upset': 55006,
 'upset.': 55007,
 'upsets': 55008,
 'upsetting': 55009,
 'upside': 55010,
 'upsidedown': 55011,
 'upsidedown.': 55012,
 'upstairs': 55013,
 'upstairs.': 5

 'voluptuous': 56086,
 'volvo': 56087,
 'volvo.': 56088,
 'vomit': 56089,
 'vomiting': 56090,
 'vomits': 56091,
 'von': 56092,
 'voodoo': 56093,
 'voodooritual': 56094,
 'voorhees': 56095,
 'voorhees.': 56096,
 'voories': 56097,
 'voracious': 56098,
 'vore': 56099,
 'vore.': 56100,
 'vores': 56101,
 'vorhees': 56102,
 'vornellini.': 56103,
 'vornellinimaria.': 56104,
 'vortex': 56105,
 'vosnia': 56106,
 'vosnian': 56107,
 'voss': 56108,
 'vossie': 56109,
 'vossie.': 56110,
 'vossoughi': 56111,
 'vosss': 56112,
 'vote': 56113,
 'voted': 56114,
 'voters': 56115,
 'votes': 56116,
 'voting': 56117,
 'vou': 56118,
 'vous': 56119,
 'vow': 56120,
 'vowed': 56121,
 'vowels': 56122,
 'vowing': 56123,
 'vows': 56124,
 'vows.': 56125,
 'voxuber': 56126,
 'voyage': 56127,
 'voyage.': 56128,
 'voyaging': 56129,
 'voyda': 56130,
 'voyda.': 56131,
 'voyeur': 56132,
 'voyeuristic': 56133,
 'voyte': 56134,
 'vr': 56135,
 'vr.': 56136,
 'vreeland': 56137,
 'vries': 56138,
 'vrit': 56139,
 'vrmland': 561

 'winner': 57298,
 'winner.': 57299,
 'winners': 57300,
 'winners.': 57301,
 'winnertakeall': 57302,
 'winnetka': 57303,
 'winnetou': 57304,
 'winnetouch': 57305,
 'winnie': 57306,
 'winnies': 57307,
 'winnifred': 57308,
 'winning': 57309,
 'winning.': 57310,
 'winninger': 57311,
 'winnings': 57312,
 'winnings.': 57313,
 'winos': 57314,
 'wins': 57315,
 'wins.': 57316,
 'winslow': 57317,
 'winson': 57318,
 'winsor': 57319,
 'winstead': 57320,
 'winston': 57321,
 'winstone': 57322,
 'winter': 57323,
 'winter.': 57324,
 'winterbottom': 57325,
 'winterbourne': 57326,
 'winterbourne.': 57327,
 'winterbournes': 57328,
 'wintercovered': 57329,
 'winternight': 57330,
 'winters': 57331,
 'winters.': 57332,
 'wintersborn': 57333,
 'wintery': 57334,
 'winthrop.': 57335,
 'winton': 57336,
 'wintry': 57337,
 'wipe': 57338,
 'wiped': 57339,
 'wipes': 57340,
 'wire': 57341,
 'wire.': 57342,
 'wired': 57343,
 'wireone': 57344,
 'wires': 57345,
 'wiretapping': 57346,
 'wiretaps': 57347,
 'wirewalker':

In [136]:
# vocabulary for targets
t_vocab = list(set(sum(targets, [])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}

pprint(target2idx)

{'.': 3,
 '..': 4,
 '...': 5,
 '....': 6,
 '.....': 7,
 '......': 8,
 '...a': 9,
 '...all': 10,
 '...and': 11,
 '...before': 12,
 '...being': 13,
 '...bring': 14,
 '...burning': 15,
 '...but': 16,
 '...cause': 17,
 '...each': 18,
 '...even': 19,
 '...fall': 20,
 '...from': 21,
 '...he': 22,
 '...hers': 23,
 '...hes': 24,
 '...high': 25,
 '...his': 26,
 '...i': 27,
 '...ill': 28,
 '...in': 29,
 '...is': 30,
 '...it': 31,
 '...its': 32,
 '...just': 33,
 '...justice': 34,
 '...lies': 35,
 '...meets': 36,
 '...men': 37,
 '...much': 38,
 '...no': 39,
 '...now': 40,
 '...or': 41,
 '...ora': 42,
 '...playing': 43,
 '...she': 44,
 '...slavery': 45,
 '...so': 46,
 '...starts': 47,
 '...street': 48,
 '...suddenly': 49,
 '...that': 50,
 '...the': 51,
 '...then': 52,
 '...there': 53,
 '...they': 54,
 '...theyre': 55,
 '...to': 56,
 '...together': 57,
 '...wait': 58,
 '...welcome': 59,
 '...what': 60,
 '...when': 61,
 '...with': 62,
 '...writing': 63,
 '...your': 64,
 '..told': 65,
 '.45caliber': 6

 'bend': 1421,
 'bending': 1422,
 'beneath': 1423,
 'beneath...': 1424,
 'benedicts': 1425,
 'benefits': 1426,
 'benjamin': 1427,
 'benjis': 1428,
 'bennies': 1429,
 'bent': 1430,
 'berault': 1431,
 'bereiden': 1432,
 'berg': 1433,
 'bergenmccarthy': 1434,
 'bergman': 1435,
 'bergner': 1436,
 'berkeley': 1437,
 'berleing': 1438,
 'berlin': 1439,
 'berlin.': 1440,
 'berlin...': 1441,
 'berlins': 1442,
 'bernard': 1443,
 'bernhardt.': 1444,
 'bernice': 1445,
 'bernie': 1446,
 'bersama': 1447,
 'berttelse': 1448,
 'berwind': 1449,
 'best': 1450,
 'best.': 1451,
 'best...': 1452,
 'best...because': 1453,
 'bestest': 1454,
 'bestkept': 1455,
 'bestseller': 1456,
 'bestseller...the': 1457,
 'bestselling': 1458,
 'bet': 1459,
 'betaler': 1460,
 'beten': 1461,
 'beth': 1462,
 'betray': 1463,
 'betray.': 1464,
 'betray...duty': 1465,
 'betrayal': 1466,
 'betrayal.': 1467,
 'betrayal..': 1468,
 'betrayed': 1469,
 'betrayed.': 1470,
 'bets': 1471,
 'bets.': 1472,
 'bette': 1473,
 'better': 1474,


 'cold.': 2624,
 'coldblooded': 2625,
 'colder': 2626,
 'coldest': 2627,
 'cole': 2628,
 'collaborate': 2629,
 'collaboration': 2630,
 'collaboration.': 2631,
 'collaborative': 2632,
 'collection': 2633,
 'collective': 2634,
 'collector': 2635,
 'collects': 2636,
 'college': 2637,
 'collegeno': 2638,
 'collide': 2639,
 'collide.': 2640,
 'collided': 2641,
 'collins': 2642,
 'collision': 2643,
 'colman': 2644,
 'colonel': 2645,
 'color': 2646,
 'color.': 2647,
 'colored': 2648,
 'colorful': 2649,
 'colors': 2650,
 'colorsone': 2651,
 'colour': 2652,
 'colpire': 2653,
 'colpisce': 2654,
 'columbia': 2655,
 'colvilleshe': 2656,
 'coma.': 2657,
 'combat': 2658,
 'combatieron': 2659,
 'combination': 2660,
 'combine...modern': 2661,
 'combined': 2662,
 'combustion.': 2663,
 'comdia': 2664,
 'come': 2665,
 'come.': 2666,
 'come...': 2667,
 'comeback': 2668,
 'comeback.': 2669,
 'comedian': 2670,
 'comedians...one': 2671,
 'comedy': 2672,
 'comedy.': 2673,
 'comedy...': 2674,
 'comedy...as': 2

 'easy.': 3985,
 'easy...': 3986,
 'eat': 3987,
 'eaten': 3988,
 'eaten.': 3989,
 'eating': 3990,
 'eats': 3991,
 'eberhardt.': 3992,
 'ech': 3993,
 'echo': 3994,
 'echoes': 3995,
 'eclectic': 3996,
 'economic': 3997,
 'ecstasy': 3998,
 'ed': 3999,
 'eddie': 4000,
 'eddies': 4001,
 'eden': 4002,
 'edge': 4003,
 'edge.': 4004,
 'editor': 4005,
 'edmond': 4006,
 'edmonton': 4007,
 'educated': 4008,
 'education': 4009,
 'edward': 4010,
 'edwards': 4011,
 'een': 4012,
 'eerie': 4013,
 'effect': 4014,
 'effect...': 4015,
 'effects': 4016,
 'effects.': 4017,
 'effekter': 4018,
 'effort': 4019,
 'efforts': 4020,
 'efter': 4021,
 'egentligen.': 4022,
 'egg': 4023,
 'eggs...': 4024,
 'egy': 4025,
 'egypt': 4026,
 'ehrgeizigen': 4027,
 'eiffel': 4028,
 'eight': 4029,
 'eight.': 4030,
 'eighth': 4031,
 'eightyearolds': 4032,
 'ein': 4033,
 'eine': 4034,
 'einer': 4035,
 'eines': 4036,
 'either': 4037,
 'ek': 4038,
 'ekberg': 4039,
 'ekmans': 4040,
 'el': 4041,
 'elder': 4042,
 'election': 4043,
 

 'globe': 5371,
 'glorified': 5372,
 'glorifies': 5373,
 'glorifying': 5374,
 'glorious': 5375,
 'glory': 5376,
 'glory.': 5377,
 'glory....it': 5378,
 'glory...a': 5379,
 'glory...and': 5380,
 'glory...it': 5381,
 'glossy': 5382,
 'glowing': 5383,
 'gman': 5384,
 'gmen...until': 5385,
 'gnter': 5386,
 'gnulegeips': 5387,
 'go': 5388,
 'go.': 5389,
 'go...': 5390,
 'go...and': 5391,
 'go...for...your': 5392,
 'go...to': 5393,
 'goal': 5394,
 'goal.': 5395,
 'goals': 5396,
 'goats': 5397,
 'gob': 5398,
 'gobi': 5399,
 'gobs': 5400,
 'god': 5401,
 'god.': 5402,
 'god...or': 5403,
 'god...the': 5404,
 'goddag': 5405,
 'goddess': 5406,
 'goddessruler': 5407,
 'godfather': 5408,
 'godly': 5409,
 'gods': 5410,
 'gods.': 5411,
 'goed': 5412,
 'goeds': 5413,
 'goes': 5414,
 'goethe': 5415,
 'goforbroke': 5416,
 'gogetter': 5417,
 'goggomobil': 5418,
 'gogo': 5419,
 'going': 5420,
 'going.': 5421,
 'gold': 5422,
 'gold.': 5423,
 'gold...': 5424,
 'gold...and': 5425,
 'goldbergs': 5426,
 'goldbr

...

 'oser': 9120,
 'oss': 9121,
 'otar': 9122,
 'other': 9123,
 'other.': 9124,
 'other...': 9125,
 'others': 9126,
 'others.': 9127,
 'otilia': 9128,
 'otley...well': 9129,
 'otoole': 9130,
 'otoole.': 9131,
 'otstupat': 9132,
 'ought': 9133,
 'our': 9134,
 'ours': 9135,
 'ours.': 9136,
 'ourselves': 9137,
 'ourselves.': 9138,
 'out': 9139,
 'out.': 9140,
 'out...': 9141,
 'out...and': 9142,
 'out...the': 9143,
 'out.and': 9144,
 'outa': 9145,
 'outcast': 9146,
 'outdoor': 9147,
 'outdoors': 9148,
 'outer': 9149,
 'outfight': 9150,
 'outfit': 9151,
 'outforever.': 9152,
 'outing.': 9153,
 'outlaw': 9154,
 'outlaws': 9155,
 'outlaws.': 9156,
 'outlaws...sweetheart': 9157,
 'outlaws...with': 9158,
 'outlawskate': 9159,
 'outlive': 9160,
 'outrageous': 9161,
 'outrageously': 9162,
 'outrages': 9163,
 'outshoot': 9164,
 'outside': 9165,
 'outside...': 9166,
 'outsider': 9167,
 'outstanding': 9168,
 'outthe': 9169,
 'outwhips': 9170,
 'outwhy': 9171,
 'over': 9172,
 'over.': 9173,
 'over...':

 'republicanplattsburg': 10370,
 'republics': 10371,
 'repulsion': 10372,
 'repunlik': 10373,
 'reputation.': 10374,
 'requiem': 10375,
 'requires': 10376,
 'rerelaseposterall': 10377,
 'rerelease': 10378,
 'rescue': 10379,
 'rescue.': 10380,
 'rescues': 10381,
 'research.': 10382,
 'resemblance': 10383,
 'reservation': 10384,
 'reserved': 10385,
 'reshaping': 10386,
 'resides': 10387,
 'resist': 10388,
 'resist.': 10389,
 'resist...': 10390,
 'resistance': 10391,
 'resisted.': 10392,
 'resolve.': 10393,
 'resorts': 10394,
 'resounding': 10395,
 'respect': 10396,
 'respectability': 10397,
 'respondi': 10398,
 'responsibilities': 10399,
 'responsibility...': 10400,
 'responsible': 10401,
 'rest': 10402,
 'rest.': 10403,
 'restaurant': 10404,
 'restless': 10405,
 'restless.': 10406,
 'restoring': 10407,
 'restricted.': 10408,
 'rests': 10409,
 'rests.': 10410,
 'result': 10411,
 'resultar': 10412,
 'results': 10413,
 'resumed': 10414,
 'resurrection...': 10415,
 'retired': 10416,
 'retni

 'smart.': 11620,
 'smart...too': 11621,
 'smartest': 11622,
 'smash': 11623,
 'smashed': 11624,
 'smashes': 11625,
 'smashing': 11626,
 'smell': 11627,
 'smells': 11628,
 'smerte': 11629,
 'smhandlare': 11630,
 'smile': 11631,
 'smile.': 11632,
 'smiles': 11633,
 'smiling': 11634,
 'smith': 11635,
 'smithtown': 11636,
 'smitten': 11637,
 'smoke': 11638,
 'smokes': 11639,
 'smokey': 11640,
 'smoking': 11641,
 'smoldering': 11642,
 'smooth': 11643,
 'smothered': 11644,
 'smuggled': 11645,
 'smuggler': 11646,
 'smugglers': 11647,
 'smuggling....': 11648,
 'smurfing': 11649,
 'smut': 11650,
 'snails': 11651,
 'snake': 11652,
 'snakes': 11653,
 'snapper.': 11654,
 'snaps': 11655,
 'snatched': 11656,
 'sneaky': 11657,
 'snitch': 11658,
 'snobs': 11659,
 'snookie': 11660,
 'snow': 11661,
 'snow.': 11662,
 'snowboarding': 11663,
 'snowman.': 11664,
 'snowy': 11665,
 'snuffed': 11666,
 'snurretvlingar': 11667,
 'so': 11668,
 'so.': 11669,
 'soaked': 11670,
 'soap': 11671,
 'soar.': 11672,
 'so

 'timreallife': 12870,
 'tims': 12871,
 'tina': 12872,
 'tingle': 12873,
 'tingles': 12874,
 'tinhorn': 12875,
 'tinstar': 12876,
 'tip': 12877,
 'tips': 12878,
 'tire': 12879,
 'tired': 12880,
 'tired.': 12881,
 'tirescu': 12882,
 'tis': 12883,
 'titanic': 12884,
 'titanic.': 12885,
 'titans.': 12886,
 'title': 12887,
 'tjaden': 12888,
 'tjejer': 12889,
 'tjorven': 12890,
 'tkak': 12891,
 'tll': 12892,
 'tnterror': 12893,
 'to': 12894,
 'to.': 12895,
 'to...': 12896,
 'to...except': 12897,
 'toast.': 12898,
 'tobi': 12899,
 'tobias': 12900,
 'tobruk': 12901,
 'tobys': 12902,
 'tod': 12903,
 'tod.': 12904,
 'toda': 12905,
 'today': 12906,
 'today.': 12907,
 'today...and': 12908,
 'todays': 12909,
 'todennkinen': 12910,
 'todolist': 12911,
 'todos': 12912,
 'toes': 12913,
 'toes...': 12914,
 'together': 12915,
 'together.': 12916,
 'together...': 12917,
 'together...and': 12918,
 'together...at': 12919,
 'togetheragainst': 12920,
 'togetherbut': 12921,
 'togethernobody': 12922,
 'togs':

 'wind': 14119,
 'windbag': 14120,
 'winding': 14121,
 'window': 14122,
 'window.': 14123,
 'windows': 14124,
 'winds': 14125,
 'windsor': 14126,
 'wine': 14127,
 'winemaker': 14128,
 'winging': 14129,
 'wings': 14130,
 'wings.': 14131,
 'winner': 14132,
 'winner...': 14133,
 'winners': 14134,
 'winners.': 14135,
 'winning': 14136,
 'wins': 14137,
 'winter': 14138,
 'winter.': 14139,
 'winthrop': 14140,
 'wipe': 14141,
 'wiped': 14142,
 'wire': 14143,
 'wires': 14144,
 'wirklichkeit': 14145,
 'wisc.': 14146,
 'wisdom': 14147,
 'wisdom.': 14148,
 'wise': 14149,
 'wiseguys.': 14150,
 'wiselybut': 14151,
 'wish': 14152,
 'wish.': 14153,
 'wished': 14154,
 'wishes': 14155,
 'wishing': 14156,
 'wisp': 14157,
 'wit.': 14158,
 'witch': 14159,
 'witchcraft': 14160,
 'witches': 14161,
 'witching': 14162,
 'witchs': 14163,
 'with': 14164,
 'with.': 14165,
 'with...': 14166,
 'with...original': 14167,
 'with...the': 14168,
 'wither': 14169,
 'withers.': 14170,
 'within': 14171,
 'within.': 14172,

In [185]:
s_input = list(map(lambda sentence : [source2idx.get(token) for token in sentence], sources))
s_input = pad_sequences(sequences = s_input, maxlen = s_max_len, padding = 'post', truncating = 'post')
s_input

array([[20480, 51798, 21903, ..., 54486, 10110, 41888],
       [ 1775,   985, 48902, ..., 54699, 53393,  3723],
       [  985, 48285, 51726, ..., 36981, 38389, 36932],
       ...,
       [22765, 57395,   985, ..., 50758,   985, 10338],
       [52170, 22265, 50308, ..., 16664, 10212, 19608],
       [  985,  5178, 17455, ..., 49320, 51735, 26034]])

In [188]:
t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], targets))
t_input = list(map(lambda sentence : [target2idx.get(token) for token in sentence], t_input))
t_input

[[1, 12642, 8729, 2329, 2],
 [1, 6746, 651, 12642, 321, 9325, 2],
 [1, 324, 8237, 218, 8194, 14420, 1130, 13483, 2],
 [1, 13925, 666, 5840, 9134, 11498, 2],
 [1,
  12689,
  900,
  12642,
  3250,
  8950,
  3228,
  12690,
  900,
  13792,
  4897,
  14398,
  12785,
  9008,
  1618,
  2],
 [1,
  12642,
  5637,
  14057,
  7458,
  5856,
  5675,
  10892,
  4911,
  6089,
  12894,
  11942,
  6502,
  12642,
  9346,
  8950,
  333,
  1963,
  4897,
  331,
  1157,
  2],
 [1, 6746, 8829, 6960, 331, 7893, 14254, 845, 2],
 [1, 4284, 6909, 8539, 1356, 14164, 9017, 11615, 12012, 2],
 [1, 14371, 8713, 7123, 14056, 5420, 12894, 1284, 14390, 13804, 2066, 2],
 [1, 3309, 6721, 725, 12642, 4122, 2],
 [1, 689, 257, 12703, 9103, 9721, 2],
 [1, 14371, 2120, 10732, 5071, 331, 8761, 2],
 [1, 5942, 5828, 8785, 5133, 2],
 [1, 7547, 7458, 7496, 14371, 8029, 6728, 2],
 [1, 11148, 11724, 1317, 2],
 [1,
  6027,
  770,
  6530,
  7848,
  5138,
  14164,
  9769,
  1486,
  2735,
  770,
  13465,
  12609,
  14042,
  6502,
  9017,

In [171]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    
    if mode == 'source':
        # preprocessing for source (encoder)
        #각 summary에 있는 단어들을 인덱스로 변환.
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        s_len = list(map(lambda sentence : len(sentence), s_input))
        #길이가 같지 않은 데이터들을 일정한 길이로 맞춰 준다.
        #padding ='post' -> 각 시퀀스 뒤쪽으로 패딩한다
        #maxlen -> 최대 길이를 max_len으로 설정하고 초과한 경우 각 시퀀스의 뒷쪽에서 자른다.
        #summary별 단어 수가 다를것. 이를 맞춰주는 역할이 pad_sequences. 단어 수가 max_len보다 적거나 클경우
        #뒷쪽부터 차이만큼 제거
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input
    
    elif mode == 'target':
        # preprocessing for target (decoder)
        # 각 태그라인 앞 뒤에 <bos>와 <eos>를 추가
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
         
        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
        
        return t_len, t_input, t_output

122.84182491894396

9.635942566002779

In [139]:
sum = 0
for i in df.movie_summary:
    length = len(i.split())
    sum += length
average_no = sum / len(df) #summary당 평균 122.84 단어
# preprocessing for source
#단어 수는 120개로 제한
s_max_len = 120
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
print(s_len, s_input)

[109, 346, 386, 109, 292, 56, 90, 200, 463, 25, 209, 104, 270, 231, 303, 230, 343, 179, 199, 406, 250, 561, 125, 681, 61, 181, 64, 187, 111, 162, 216, 54, 190, 94, 141, 25, 105, 70, 39, 189, 224, 55, 480, 89, 455, 73, 189, 279, 154, 18, 152, 128, 173, 34, 549, 205, 184, 80, 87, 62, 133, 58, 77, 62, 68, 104, 64, 216, 17, 17, 18, 150, 173, 332, 218, 134, 188, 129, 87, 475, 25, 101, 322, 223, 204, 189, 81, 117, 50, 248, 73, 321, 256, 219, 85, 286, 22, 99, 72, 374, 30, 199, 21, 198, 124, 113, 35, 108, 60, 79, 182, 280, 125, 402, 206, 50, 177, 35, 128, 102, 117, 33, 173, 248, 330, 79, 219, 672, 97, 116, 130, 53, 230, 49, 125, 24, 208, 174, 188, 102, 228, 98, 98, 79, 24, 186, 141, 404, 59, 162, 535, 174, 62, 107, 306, 35, 157, 415, 494, 122, 41, 128, 50, 46, 183, 225, 77, 64, 40, 716, 83, 182, 183, 111, 46, 165, 61, 133, 113, 172, 296, 310, 107, 268, 164, 77, 96, 342, 132, 150, 94, 269, 137, 668, 103, 153, 89, 32, 411, 185, 206, 111, 130, 131, 63, 322, 66, 75, 126, 892, 80, 169, 155, 179, 39

In [140]:
sum = 0
for i in df.movie_tagline:
    length = len(i.split())
    sum += length
average_no = sum / len(df) #각 tagline별 단어 수는 9.6개

# preprocessing for target
#단어 수 10개로 제한
t_max_len = 10
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)

[5, 7, 9, 7, 16, 22, 9, 10, 12, 7, 7, 8, 6, 8, 5, 28, 5, 10, 5, 8, 4, 9, 21, 16, 19, 9, 6, 8, 7, 8, 14, 10, 12, 6, 9, 11, 6, 15, 10, 19, 12, 7, 6, 5, 26, 9, 14, 7, 4, 5, 8, 5, 13, 12, 10, 7, 14, 12, 9, 8, 7, 7, 21, 7, 9, 14, 6, 16, 8, 6, 19, 10, 8, 12, 6, 10, 7, 16, 14, 11, 16, 7, 8, 16, 13, 7, 7, 8, 13, 13, 8, 10, 8, 8, 19, 36, 13, 10, 10, 6, 8, 6, 14, 9, 8, 11, 12, 10, 9, 12, 6, 8, 9, 6, 9, 7, 5, 5, 9, 12, 8, 28, 14, 12, 10, 10, 14, 5, 13, 9, 5, 5, 20, 13, 10, 10, 16, 9, 4, 8, 15, 10, 15, 63, 11, 8, 17, 11, 10, 9, 12, 6, 5, 18, 5, 5, 9, 26, 8, 11, 11, 7, 14, 13, 9, 7, 7, 7, 7, 6, 3, 5, 6, 10, 9, 14, 5, 7, 12, 7, 10, 19, 13, 12, 13, 6, 7, 6, 12, 6, 9, 6, 11, 8, 29, 34, 10, 9, 14, 6, 19, 10, 8, 8, 8, 6, 20, 8, 20, 8, 15, 4, 7, 8, 7, 7, 11, 8, 15, 11, 13, 15, 8, 8, 5, 21, 12, 5, 11, 7, 23, 5, 11, 12, 11, 9, 11, 6, 8, 7, 8, 8, 11, 7, 9, 9, 10, 10, 37, 13, 13, 12, 10, 11, 11, 9, 13, 10, 23, 7, 20, 22, 17, 5, 8, 9, 7, 10, 9, 6, 23, 7, 7, 7, 23, 5, 25, 8, 34, 11, 6, 11, 15, 18, 11, 5, 11, 5

 [[    1 12642  8729 ...     0     0     0]
 [    1  6746   651 ...     0     0     0]
 [    1   324  8237 ...     0     0     0]
 ...
 [    1 12642  8411 ...     0     0     0]
 [    1 12729  8456 ...  8234     2     0]
 [    1 13289 10547 ...  1001 12642  8970]] [[12642  8729  2329 ...     0     0     0]
 [ 6746   651 12642 ...     0     0     0]
 [  324  8237   218 ...     0     0     0]
 ...
 [12642  8411  3190 ...     0     0     0]
 [12729  8456  1342 ...     2     0     0]
 [13289 10547 12518 ... 12642  8970     2]]


# hyper-param

In [141]:
# hyper-parameters
epochs = 100
batch_size = 4
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units = 128

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [142]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [143]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
#         print("state: {}".format(state.shape))
#         print("output: {}".format(state.shape))
              
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [144]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        # * `score = FC(tanh(FC(EO) + FC(H)))`
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
                
        #* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, 1)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        # * `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        # * `embedding output` = The input to the decoder X is passed through an embedding layer.
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        # * `merged vector = concat(embedding output, context vector)`
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [146]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    
#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))
    
    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints_attention_withpreprocesseddata'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)

# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir=checkpoint_dir)

In [148]:
EPOCHS = 200

for epoch in range(EPOCHS):
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(s_input, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #Teacher Forcing: feeding the target as the next input
            for t in range(1, t_input.shape[1]):
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(t_input[:, t], predictions)
            
                dec_input = tf.expand_dims(t_input[:, t], 1) #using teacher forcing
                
        batch_loss = (loss / int(t_input.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradient = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradient, variables))
        
    if epoch % 10 == 0:
        #save model every 10 epoch
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                            total_loss / n_batch,
                                            batch_loss.numpy()))
        checkpoint.save(file_prefix = checkpoint_prefix)

Epoch 0 Loss 342.5703 Batch Loss 4.0285
Epoch 10 Loss 213.7034 Batch Loss 2.8999
Epoch 20 Loss 143.5222 Batch Loss 2.3344
Epoch 30 Loss 112.4594 Batch Loss 1.9094
Epoch 40 Loss 97.3844 Batch Loss 1.7990
Epoch 50 Loss 66.7112 Batch Loss 1.5354
Epoch 60 Loss 50.2577 Batch Loss 0.8989
Epoch 70 Loss 39.2490 Batch Loss 0.6431
Epoch 80 Loss 31.3291 Batch Loss 0.6616
Epoch 90 Loss 29.2967 Batch Loss 0.3746
Epoch 100 Loss 24.0294 Batch Loss 0.3169
Epoch 110 Loss 16.6932 Batch Loss 0.3443
Epoch 120 Loss 31.5051 Batch Loss 0.8361
Epoch 130 Loss 27.7123 Batch Loss 0.3194
Epoch 140 Loss 15.8377 Batch Loss 0.2449
Epoch 150 Loss 13.1056 Batch Loss 0.1663
Epoch 160 Loss 10.5683 Batch Loss 0.2059
Epoch 170 Loss 9.2908 Batch Loss 0.1245
Epoch 180 Loss 8.7706 Batch Loss 0.1005
Epoch 190 Loss 6.7926 Batch Loss 0.1251


In [149]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# result, sentence, attention_plot = evaluate(sentence, encoder, decoder, source2idx, target2idx,
#                                             s_max_len, t_max_len)

In [150]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [151]:

def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    
#     attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
#     plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [152]:
#restore checkpoint

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [153]:
sentence = df.movie_summary[1]


In [154]:
sentence

'after a space merchant vessel perceives an unknown transmission as a distress call its landing on the source moon finds one of the crew attacked by a mysterious lifeform and they soon realize that its life cycle has merely begun. in the distant future the crew of the commercial spaceship nostromo are on their way home when they pick up a distress call from a distant moon. the crew are under obligation to investigate and the spaceship descends on the moon afterwards. after a rough landing three crew members leave the spaceship to explore the area on the moon. at the same time as they discover a hive colony of some unknown creature the ships computer deciphers the message to be a warning not a distress call. when one of the eggs is disturbed the crew realizes that they are not alone on the spaceship and they must deal with the consequences. in the far reaches of space a crew onboard the ship nostromo are returning to earth until they receive a distress call an alien one. unknown to them

In [155]:

# sentence = 'tensorflow is a framework for deep learning'

translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

Input: after a space merchant vessel perceives an unknown transmission as a distress call its landing on the source moon finds one of the crew attacked by a mysterious lifeform and they soon realize that its life cycle has merely begun. in the distant future the crew of the commercial spaceship nostromo are on their way home when they pick up a distress call from a distant moon. the crew are under obligation to investigate and the spaceship descends on the moon afterwards. after a rough landing three crew members leave the spaceship to explore the area on the moon. at the same time as they discover a hive colony of some unknown creature the ships computer deciphers the message to be a warning not a distress call. when one of the eggs is disturbed the crew realizes that they are not alone on the spaceship and they must deal with the consequences. in the far reaches of space a crew onboard the ship nostromo are returning to earth until they receive a distress call an alien one. unknown t

In [159]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    xList = []
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()
        x = tf.nn.top_k(predictions[0], k= 3, sorted=True, name = None)
        for i in range(3):
            predicted_id = x[1][i].numpy()

            result += idx2target[predicted_id] + ' '

            if idx2target.get(predicted_id) == '<eos>':
                return result, sentence, attention_plot

            # the predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)
        xList.append([result])
        result=''

    return xList

tList = evaluate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
from pprint import pprint
pprint(tList)

# result, sentence, attention_plot = evaluate(sentence, encoder, decoder, source2idx, target2idx,
#                                             s_max_len, t_max_len)

[['the you when '],
 ['a the you '],
 ['can cant going '],
 ['to from about '],
 ['to people love '],
 ['is tough doesnt '],
 ['kill have happen '],
 ['to in when '],
 ['you the a '],
 ['miracle hero number '],
 ['who two story '],
 ['has and is ']]


In [94]:
df[df['movie_tagline'].str.contains('a new comedy')]

,level_0,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
6049,6442,19810,[Comedy],Nearly a Nasty Accident,www.imdb.com/title/tt0055221/mediaviewer/rm233...,the raf group captain has a hard job to restra...,"[\n , \n ]",that daffadilly blonde from carry on nurse in ...
8334,85,178,"[Comedy, Romance]",She's All That,www.imdb.com/title/tt0160862/mediaviewer/rm296...,a high school jock makes a bet that he can tur...,"[\n , \n ]",a new comedy that proves theres more to attrac...


In [96]:
df.movie_tagline[8334]

'a new comedy that proves theres more to attraction than meets the eye.'

In [85]:
sentence = df.movie_summary[106].split()[:15]
sentence = ' '.join(sentence)

In [86]:
sentence

'a group of bostonbred gangsters set up shop in balmy florida during the prohibition era'

In [91]:
translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

Input: a group of bostonbred gangsters set up shop in balmy florida during the prohibition era
Predicted translation: The master of the most important and powerful films of the most 


In [110]:
sentence = df.movie_summary[107].split()[:30]
sentence = ' '.join(sentence)
sentence

'an indepth examination of the ways in which the us vietnam war impacts and disrupts the lives of people in a small industrial town in pennsylvania michael steven and nick'

In [111]:
translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

Input: an indepth examination of the ways in which the us vietnam war impacts and disrupts the lives of people in a small industrial town in pennsylvania michael steven and nick
Predicted translation: The Past Never Stays in Focus. <eos> 


In [112]:
df[df['movie_tagline'].str.contains('Past Never Stays in Focus')]

,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
108,"['Drama', 'Mystery', 'Romance']",The Souvenir,www.imdb.com/title/tt6920356/mediaviewer/rm353...,a young film student in the early 80s becomes ...,Julie Honor Swinton Byrne is a young film stud...,The Past Never Stays in Focus.


In [108]:
df.movie_tagline[107]

'One of the most important and powerful films of all time!'

In [109]:
df.movie_summary[154]

'selfish yuppie charlie babbitts father left a fortune to his savant brother raymond and a pittance to charlie they travel crosscountry charles sanford charlie babbit is a selfcentered los angelesbased automobile dealerhustlerbookie who is at war with his own life charlie as a young teenager used his fathers 1949 buick convertible without permission and as a result he went to jail for two days on account that his father reported it stolen it is then that charlie learns that his estranged father died and left him from his last will and testament a huge bed of roses and the car while the remainder will of 3 million goes into a trust fund to be distributed to someone charlie seemed pretty angry by this and decides to look into this matter it seems as if that someone is raymond charlies unknown brother an autistic savant who lives in a world of his own resides at the walbrook institute charlie then kidnaps raymond and decides to take him on a lust for life trip to the west coast as a threa

In [82]:
sentence = 'will persist in the gulf It’s unlikely to end until the excess capacity in real estate has been cleared'
sentence

'will persist in the gulf It’s unlikely to end until the excess capacity in real estate has been cleared'

In [83]:
translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

Input: will persist in the gulf It’s unlikely to end until the excess capacity in real estate has been cleared
Predicted translation: a daughter a father go to run like paradise like paradise like 
